<a href="https://colab.research.google.com/github/BlueBerry-Coder/Practice/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
from glob import glob
import matplotlib.pylab as plt
import pydicom as dicom
import nibabel as nib
from sklearn.preprocessing import MinMaxScaler
from skimage.transform import rotate

In [ ]:
TRAINING_DATA_PATH = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
VALIDATION_DATA_PATH = '../input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'

    # Training Data,
        #t1-исходные, t1ce-постконтрастные взвешенные, t2-взвешенные, flair-с подавленной инверсией восстановлением жидкости, seg-сегментированные

flair_image_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_003/BraTS20_Training_003_flair.nii').get_fdata()
mask_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_003/BraTS20_Training_003_seg.nii').get_fdata()
t1_image_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_003/BraTS20_Training_003_t1.nii').get_fdata()
t1ce_image_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_003/BraTS20_Training_003_t1ce.nii').get_fdata()
t2_image_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_003/BraTS20_Training_003_t2.nii').get_fdata()

# path = 'D:/PyCode/Project/data/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t2.nii'

    # Validation Data
        #t1-исходные, t1ce-постконтрастные взвешенные, t2-взвешенные, flair-с подавленной инверсией восстановлением жидкости

# path = 'D:/PyCode/Project/data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_flair.nii'
# path = 'D:/PyCode/Project/data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_t1.nii'
# path = 'D:/PyCode/Project/data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_t1ce.nii'
# path = 'D:/PyCode/Project/data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_t2.nii'

In [ ]:
class BratsDataset(Dataset):
    def __init__(self, t1_image_dir, mask_dir, transform=None):
        self.t1_image_dir = t1_image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(t1_image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.t1_image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask